# 编译 MMDeploy

In [ ]:
#安装MMDEPLOY 
git clone https://github.com/open-mmlab/mmdeploy.git
cd mmdeploy
os.environ['MMDEPLOY_DIR']='/kaggle/working/mmdeploy'
pip install -e .

In [ ]:
#编译自定义算子库，写完新算子要重新编译一次,这个也可以用
mkdir -p build
cd build
cmake -DCMAKE_CXX_COMPILER=g++ -DMMDEPLOY_TARGET_BACKENDS=ort -DONNXRUNTIME_DIR=${ONNXRUNTIME_DIR} ..
make -j$(nproc) 
make install
#.so文件就在Lib文件夹下

# 部署

In [ ]:
%cd /kaggle/working/
!git clone https://github.com/open-mmlab/mmclassification.git
%cd mmclassification
!pip install -e . 
#保存路径
!mkdir checkpoints
#下载checkpoint

!wget -c https://download.openmmlab.com/mmclassification/v0/vit/pretrain/vit-base-p16_3rdparty_pt-64xb64_in1k-224_20210928-02284250.pth \
      -O checkpoints/tmp.pth

In [ ]:
!python ./tools/deploy.py \
    #这个是config里面的文件，看自己的训练要求看看要不要重写
    ./configs/mmcls/classification_onnxruntime_static.py \
    #这个是模型文件
    /kaggle/working/mmclassification/configs/resnet/resnet34_b16x8_cifar10.py \
    #这个是权重
    /kaggle/working/mmclassification/checkpoints/tmp.pth \
    # 模型转换时，用做测试的图像或点云文件路径
    /kaggle/working/mmclassification/demo/dog.jpg \
    --show \
    #默认cpu 对于TRT cuda:0
    --device 'cpu'

In [ ]:
import mmcv
from PIL import Image
import onnxruntime
import torchvision.transforms as transforms
from mmcls.datasets import ImageNet
import numpy as np
pic_path='/kaggle/working/mmclassification/demo/bird.JPEG'

#变形
img=Image.open(pic_path)
img_norm_cfg = dict(mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
resize = transforms.Resize([384,384])
norm = transforms.Normalize(mean=img_norm_cfg['mean'],std=img_norm_cfg['std'])
img = resize(img)

#转成张量 ，预处理也可以在这里做
to_tensor = transforms.ToTensor()

img_y = to_tensor(img)
#img_y = norm(img_y)
img_y.unsqueeze_(0)

In [ ]:
#执行推断
#因为是在onnxruntime上执行的，所以是将.onnx文件直接作为图传入，形成session
ort_session = onnxruntime.InferenceSession('end2end.onnx')

#这个是将输入放到正确的设备上
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction执行的方法
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img_y)}#输入{'name':输入张量}
ort_outs = ort_session.run(None, ort_inputs)#输出，其后要考虑对其进行后处理，生成最终的预测结果

In [ ]:
label = ImageNet.CLASSES
prediction = np.argmax(ort_outs[0],axis=1)
pre=label[int(prediction)]
print(pre)

# torch2onnx  
   
   ### 二选一

In [ ]:
from mmdeploy.apis import torch2onnx
img = 'demo.jpg'
work_dir = 'work_dir'
save_file = 'fcos.onnx'
deploy_cfg = ('configs/mmdet/detection/detection_onnxruntime_dynamic.py')
model_cfg = ('mmdetection/configs/fcos/fcos_r50_caffe_fpn_gn-head_1x_coco.py')
#model_checkpoint = ('checkpoints/fcos_r50_caffe_fpn_gn-head_1x_coco-821213aa.pth')
#device = 'cpu'
torch2onnx(img, work_dir, save_file, deploy_cfg, model_cfg, 
          # model_checkpoint, device
          )

In [ ]:
python tools/torch2onnx.py \
    ${DEPLOY_CFG} \
    ${MODEL_CFG} \
    ${CHECKPOINT} \
    ${INPUT_IMG} \
    --work-dir ${WORK_DIR} \
    --device cpu \
    --log-level INFO

In [ ]:
#使用onnx文件转化到其他的平台，from_onnx执行命令行进行模型转化
from mmdeploy.backend.tensorrt.utils import from_onnx

engine = from_onnx(
    'srcnn3.onnx',
    'srcnn3',
    input_shapes=dict(
        input=dict(
            min_shape=[1, 3, 256, 256],
            opt_shape=[1, 3, 256, 256],
            max_shape=[1, 3, 256, 256]),
        factor=dict(
            min_shape=[1, 1, 256, 256],
            opt_shape=[1, 1, 512, 512],
            max_shape=[1, 1, 1024, 1024])))

In [ ]:
from mmengine import Config
from mmdeploy.apis.utils import build_task_processor
import mmcv
import cv2
import torch
from mmdeploy.utils import Backend, get_backend, get_input_shape
import matplotlib.pyplot as plt

In [ ]:
model_cfg= '/root/mmdetection-main/configs/fcos/fcos_r50-caffe_fpn_gn-head_1x_coco.py'
checkpoint='/root/fcos_r50_caffe_fpn_gn-head_1x_coco-821213aa.pth'
deploy_cfg = '/root/mmdeploy/configs/mmdet/detection/detection_onnxruntime_static.py'
img = '/root/mmdetection-main/demo/demo.jpg'
device='cpu'

In [ ]:
task_processor = build_task_processor(Config.fromfile(model_cfg), Config.fromfile(deploy_cfg), device)

In [ ]:
model = task_processor.build_backend_model(['/root/work_dir/FCOS.onnx'], data_preprocessor_updater=task_processor.
                update_data_preprocessor)

In [ ]:
input_shape = get_input_shape(deploy_cfg)
model_inputs, _ = task_processor.create_input(img, input_shape)
with torch.no_grad():
    result = model.test_step(model_inputs)[0]

In [ ]:
visualizer = task_processor.get_visualizer('test','/root')

In [ ]:
#如果有数据类名，要在外面把信息传递给可视化器
# `DetLocalVisualizer().dataset_meta=xxx`,
#{'classes'：[]，'palette'：[]}
img=mmcv.imread(img)
img = mmcv.imconvert(img, 'bgr', 'rgb')
visualizer.add_datasample(
    'test',
    img,
    data_sample=result,
    draw_gt=False,
    show=True,
    # out_file='/root/test.jpg'
    )

In [ ]:
img = visualizer.get_image()
plt.imshow(img)

# 自定义算子的实现

In [ ]:
#新建csrc/backend_ops/tensorrt/dynamic_resize
#添加trt_dynamic_resize.hpp和trt_dynamic_resize.cpp两个文件
#在dynamic_resize.hpp引入mmdeploy/csrc/backend_ops/tensorrt/common/trt_plugin_base.hpp

#替换了TENSORRT原始的继承基类nvinfer1::IPluginV2DynamicExt和nvinfer1::IPluginCreator
#class DynamicTRTResize : public TRTPluginBase{}
#class DynamicTRTResizeCreator : public TRTPluginCreatorBase{}

In [1]:
from ctypes import *

In [ ]:
dll = cdll.LoadLibrary(r'F:\mmdeploy-main\build\lib\Release\mmdeploy_onnxruntime_ops.lib')